In [1]:
import pandas as pd
import numpy as np
import json

# Load the training data
train_path = "/home/data/train.json"

# Read the JSON file
with open(train_path, 'r') as f:
    train_data = json.load(f)

print(f"Number of training samples: {len(train_data)}")
print(f"First sample keys: {list(train_data[0].keys())}")
print(f"\nFirst sample preview:")
print(json.dumps(train_data[0], indent=2)[:1000] + "...")

Number of training samples: 2878
First sample keys: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_pizza', 'requester_subreddits_at_request', 'requester_u

In [2]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(train_data)

# Check basic info
print("DataFrame shape:", df.shape)
print("\nTarget distribution:")
print(df['requester_received_pizza'].value_counts())
print(f"\nTarget ratio: {df['requester_received_pizza'].mean():.4f}")

# Check data types
print("\nData types:")
print(df.dtypes.head(20))

# Check for missing values
print("\nMissing values:")
print(df.isnull().sum().head(10))

DataFrame shape: (2878, 32)

Target distribution:
requester_received_pizza
False    2163
True      715
Name: count, dtype: int64

Target ratio: 0.2484

Data types:
giver_username_if_known                                  object
number_of_downvotes_of_request_at_retrieval               int64
number_of_upvotes_of_request_at_retrieval                 int64
post_was_edited                                          object
request_id                                               object
request_number_of_comments_at_retrieval                   int64
request_text                                             object
request_text_edit_aware                                  object
request_title                                            object
requester_account_age_in_days_at_request                float64
requester_account_age_in_days_at_retrieval              float64
requester_days_since_first_post_on_raop_at_request      float64
requester_days_since_first_post_on_raop_at_retrieval    float64
requ

In [4]:
# Analyze text features
print("Text feature analysis:")
print(f"Average request_text length: {df['request_text'].str.len().mean():.1f} characters")
print(f"Average request_title length: {df['request_title'].str.len().mean():.1f} characters")
print(f"Average request_text words: {df['request_text'].str.split().str.len().mean():.1f} words")

# Check unique values in categorical features
print("\nCategorical features:")
print(f"Unique requester_user_flair values: {df['requester_user_flair'].unique()}")
print(f"post_was_edited distribution:\n{df['post_was_edited'].value_counts()}")

# Check numerical feature distributions
print("\nNumerical feature statistics:")
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'requester_received_pizza' in numerical_cols:
    numerical_cols.remove('requester_received_pizza')  # Remove target
print(df[numerical_cols].describe().loc[['mean', 'std', 'min', 'max']])

Text feature analysis:
Average request_text length: 402.5 characters
Average request_title length: 71.6 characters
Average request_text words: 77.0 words

Categorical features:
Unique requester_user_flair values: [None 'shroom' 'PIF']
post_was_edited distribution:
post_was_edited
False           2423
True             241
1375324604.0       1
1366314331.0       1
1367280954.0       1
                ... 
1379372126.0       1
1378425306.0       1
1374109637.0       1
1358627245.0       1
1372729287.0       1
Name: count, Length: 216, dtype: int64

Numerical feature statistics:
      number_of_downvotes_of_request_at_retrieval  \
mean                                     2.428075   
std                                      3.035568   
min                                      0.000000   
max                                     47.000000   

      number_of_upvotes_of_request_at_retrieval  \
mean                                   6.090688   
std                                   10.501259   

In [5]:
# Check correlation with target
correlations = df[numerical_cols + ['requester_received_pizza']].corr()['requester_received_pizza'].sort_values(ascending=False)
print("Correlations with target (top 10 positive and negative):")
print(correlations.head(11))  # Including target itself
print("\n")
print(correlations.tail(10))

# Check user flair distribution with target
print("\nUser flair distribution with target:")
flair_pivot = pd.crosstab(df['requester_user_flair'], df['requester_received_pizza'], normalize='index')
print(flair_pivot)

# Check post_was_edited (clean data first - there seems to be some data quality issues)
print("\nData quality check for post_was_edited:")
print("Unique values:", df['post_was_edited'].unique()[:10])
print("Value counts (top 10):")
print(df['post_was_edited'].value_counts().head(10))

Correlations with target (top 10 positive and negative):
requester_received_pizza                                1.000000
requester_number_of_posts_on_raop_at_retrieval          0.462165
request_number_of_comments_at_retrieval                 0.290709
requester_number_of_comments_in_raop_at_retrieval       0.277129
requester_number_of_posts_on_raop_at_request            0.132712
requester_number_of_comments_in_raop_at_request         0.131965
requester_days_since_first_post_on_raop_at_retrieval    0.127262
requester_number_of_comments_at_retrieval               0.123016
requester_account_age_in_days_at_retrieval              0.118863
requester_days_since_first_post_on_raop_at_request      0.108662
number_of_upvotes_of_request_at_retrieval               0.090767
Name: requester_received_pizza, dtype: float64


requester_upvotes_plus_downvotes_at_retrieval    0.052447
requester_number_of_subreddits_at_request        0.047001
requester_account_age_in_days_at_request         0.043374
reque

In [6]:
# Clean post_was_edited - it should be boolean but contains timestamps
df['post_was_edited_clean'] = df['post_was_edited'].apply(lambda x: x if isinstance(x, bool) else True)
print("Cleaned post_was_edited distribution:")
print(df['post_was_edited_clean'].value_counts())

# Check correlation of cleaned feature
correlation_edited = df['post_was_edited_clean'].corr(df['requester_received_pizza'])
print(f"\nCorrelation of post_was_edited with target: {correlation_edited:.4f}")

# Analyze text sentiment/word patterns
import re
from collections import Counter

# Simple text features
df['text_length'] = df['request_text'].str.len()
df['title_length'] = df['request_title'].str.len()
df['text_word_count'] = df['request_text'].str.split().str.len()
df['title_word_count'] = df['request_title'].str.split().str.len()

# Check for common words in successful vs unsuccessful requests
def get_common_words(texts, n=10):
    all_text = ' '.join(texts).lower()
    words = re.findall(r'\b[a-zA-Z]{3,}\b', all_text)
    return Counter(words).most_common(n)

successful_texts = df[df['requester_received_pizza']]['request_text'].tolist()
unsuccessful_texts = df[~df['requester_received_pizza']]['request_text'].tolist()

print("\nMost common words in successful requests:")
print(get_common_words(successful_texts, 15))

print("\nMost common words in unsuccessful requests:")
print(get_common_words(unsuccessful_texts, 15))

Cleaned post_was_edited distribution:
post_was_edited_clean
False    2423
True      455
Name: count, dtype: int64

Correlation of post_was_edited with target: 0.1432

Most common words in successful requests:
[('and', 2073), ('the', 1545), ('for', 1097), ('pizza', 775), ('have', 687), ('but', 550), ('would', 530), ('this', 497), ('that', 480), ('you', 467), ('out', 408), ('with', 399), ('get', 396), ('can', 377), ('just', 368)]

Most common words in unsuccessful requests:
[('and', 4934), ('the', 3586), ('for', 2350), ('pizza', 1900), ('have', 1707), ('would', 1409), ('but', 1273), ('this', 1208), ('you', 1100), ('that', 1076), ('out', 1030), ('with', 951), ('just', 926), ('can', 872), ('get', 863)]
